In [2]:
import os
from llama_index.core import (
    KnowledgeGraphIndex, ServiceContext,
    SimpleDirectoryReader, StorageContext, Document)
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.graph_stores import SimpleGraphStore
from langchain_ollama.chat_models import ChatOllama
from dotenv import load_dotenv
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from warnings import filterwarnings
filterwarnings("ignore")

In [7]:
#setup api key
# load_dotenv()
# Initialize the 1b model for general tasks
llm = Ollama(model="gemma3:1b", 
             temperature=0.0,
             request_timeout=300,keep_alive="10m")

# Set the embedding model (Required for 'hybrid' mode)
# Make sure you have run 'ollama pull nomic-embed-text'
embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# Set these globally for LlamaIndex to use
Settings.llm = llm
Settings.embed_model = embed_model
#load documents
documents = SimpleDirectoryReader("./data").load_data()

# model = ChatOllama(
# model="gemma3:1b",
# temperature=0.5,
# )
# messages = [
#     ("system","You are a helpful agent"),
#     ("human","Hi, hello!")
# ]
# response = model.invoke(messages)
# print(response.content,end="\n")

#Initialize the Graph Store
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

#build the knowledge graph  index, this step uses LLM to extract entities and relationships
index = KnowledgeGraphIndex.from_documents(
documents,
max_triplets_per_chunk=2,
storage_context=storage_context,
include_embeddings = True,
show_progress=True,
)

# Query the Graph
query_engine = index.as_query_engine(
include_text=True,
response_mode="tree_summarize",
embedding_mode="hybrid", # uses both vector and graph search
similarity_top_k=5,
)
response = query_engine.query("Describe a cat")
print(response,end="\n")

Processing nodes:   0%|          | 0/12 [00:00<?, ?it/s]

2025-12-22 14:15:33,118 - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
2025-12-22 14:15:55,886 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
2025-12-22 14:15:56,314 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:15:56,411 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:15:56,512 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:15:56,610 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Processing nodes:   8%|▊         | 1/12 [00:23<04:20, 23.66s/it]2025-12-22 14:16:19,194 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
2025-12-22 14:16:19,315 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:16:19,402 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 1

**********
Trace: index_construction
    |_CBEventType.NODE_PARSING -> 0.02347 seconds
      |_CBEventType.CHUNKING -> 0.020712 seconds
    |_CBEventType.TEMPLATING -> 3.1e-05 seconds
    |_CBEventType.LLM -> 22.767181 seconds
    |_CBEventType.EMBEDDING -> 0.721245 seconds
    |_CBEventType.TEMPLATING -> 2.5e-05 seconds
    |_CBEventType.LLM -> 22.5841 seconds
    |_CBEventType.EMBEDDING -> 0.283967 seconds
    |_CBEventType.TEMPLATING -> 3e-05 seconds
    |_CBEventType.LLM -> 21.669465 seconds
    |_CBEventType.EMBEDDING -> 0.193459 seconds
    |_CBEventType.TEMPLATING -> 2.6e-05 seconds
    |_CBEventType.LLM -> 22.616994 seconds
    |_CBEventType.EMBEDDING -> 0.339952 seconds
    |_CBEventType.TEMPLATING -> 2.6e-05 seconds
    |_CBEventType.LLM -> 21.619256 seconds
    |_CBEventType.TEMPLATING -> 4e-05 seconds
    |_CBEventType.LLM -> 23.120903 seconds
    |_CBEventType.EMBEDDING -> 0.280436 seconds
    |_CBEventType.TEMPLATING -> 2.5e-05 seconds
    |_CBEventType.LLM -> 20.69109 se

2025-12-22 14:20:04,498 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
2025-12-22 14:20:04,552 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:20:04,555 - INFO - > Querying with idx: f365a57a-e55a-4432-8393-8a5a40fbdc72: Statistics indicate that animal lovers in recent years have shown a preferenc...
2025-12-22 14:20:04,555 - INFO - > Querying with idx: 237a05d2-004c-4347-81cb-f341364e94e6: In fact, it will give them diarrhea. Instead, give your cat cream. The butter...
2025-12-22 14:20:04,556 - INFO - > Querying with idx: 64e484f6-21d2-4697-826d-6d176cfb3fe8: There is no single cat called the panther. The name is commonly applied to th...
2025-12-22 14:20:04,557 - INFO - > Querying with idx: 96d2418d-0a81-4655-9733-b05334400f83: Cats’ hearing stops at 65 khz (kilohertz); humans’ hearing stops at 20 khz.
I...
2025-12-22 14:20:04,557 - INFO - > Querying with idx: fd1fa4ae-e733-4e86-b25f-1b1b91d22d67: Most cats h

**********
Trace: query
    |_CBEventType.QUERY -> 118.445046 seconds
      |_CBEventType.RETRIEVE -> 2.729246 seconds
        |_CBEventType.TEMPLATING -> 2.4e-05 seconds
        |_CBEventType.LLM -> 2.66929 seconds
        |_CBEventType.EMBEDDING -> 0.054163 seconds
      |_CBEventType.SYNTHESIZE -> 115.715283 seconds
        |_CBEventType.TEMPLATING -> 1.8e-05 seconds
        |_CBEventType.LLM -> 115.708862 seconds
**********
Cats have true fur, in that they have both an undercoat and an outer coat.


In [21]:

# import os
# from openai import OpenAI
# from datetime import datetime, timezone

# # Set your key in the environment before running, e.g.:
# #   export OPENAI_API_KEY="sk-..."
# # or set it here (NOT recommended to hardcode secrets)
# # os.environ["OPENAI_API_KEY"] = "sk-..."

# def list_models():
#     client = OpenAI()  # uses OPENAI_API_KEY from env
#     models = client.models.list()

#     # Convert to a friendly, sorted listing
#     items = sorted(models.data, key=lambda m: m.id)
#     print(f"Found {len(items)} models @ {datetime.now(timezone.utc).isoformat()}")
#     for m in items:
#         # Not all fields are guaranteed; guard with getattr
#         owned_by = getattr(m, "owned_by", "unknown")
#         created = getattr(m, "created", None)
#         created_iso = (
#             datetime.fromtimestamp(created, tz=timezone.utc).isoformat()
#             if isinstance(created, int) else "n/a"
#         )
#         print(f"- id: {m.id:20} | owner: {owned_by:15} | created: {created_iso}")

# if __name__ == "__main__":
#     try:
#         list_models()
#     except Exception as e:
#         print("Error listing models:", e)
#         print("• Ensure OPENAI_API_KEY is set")
#         print("• Check network access and organization/project permissions")

In [5]:

# --- Setup ---
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Optional: better logs to see what's happening
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

debug_handler = LlamaDebugHandler(print_trace_on_end=True)
cb_manager = CallbackManager([debug_handler])

# 1) Use a valid Ollama model and a sane timeout
llm = Ollama(
    model="gemma3:1b",           # or "llama3.1:8b" if you have it pulled
    request_timeout=120,         # increase for first-run warmup
    # base_url="http://localhost:11434",  # set if your Ollama runs elsewhere
)

# 2) Embeddings (hybrid needs this later anyway)
embed_model = OllamaEmbedding(model_name="nomic-embed-text")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.callback_manager = cb_manager
Settings.chunk_size = 1024  # moderate chunking

# 3) Load a small test document to verify path
documents = SimpleDirectoryReader(input_dir="./data").load_data()
# Ensure ./data has at least one small .txt/.md/.pdf

# 4) Build a vector index first (quick, reliable)
v_index = VectorStoreIndex.from_documents(documents)

# 5) Query (should return quickly and consistently)
v_engine = v_index.as_query_engine(similarity_top_k=3)
print("Vector index response:")
print(v_engine.query("Describe a cat"))


2025-12-22 14:09:48,737 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:09:52,919 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:09:57,016 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:10:00,928 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:10:04,883 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:10:08,960 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:10:12,918 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:10:16,952 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:10:21,008 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:10:25,119 - INFO - HTTP Request: POST http://localhost:1143

**********
Trace: index_construction
    |_CBEventType.NODE_PARSING -> 0.022981 seconds
      |_CBEventType.CHUNKING -> 0.021858 seconds
    |_CBEventType.EMBEDDING -> 41.057406 seconds
    |_CBEventType.EMBEDDING -> 7.758074 seconds
**********


2025-12-22 14:10:33,042 - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


Vector index response:


2025-12-22 14:10:33,103 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-12-22 14:11:31,729 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


**********
Trace: query
    |_CBEventType.QUERY -> 58.686568 seconds
      |_CBEventType.RETRIEVE -> 0.061787 seconds
        |_CBEventType.EMBEDDING -> 0.059506 seconds
      |_CBEventType.SYNTHESIZE -> 58.6242 seconds
        |_CBEventType.TEMPLATING -> 1.6e-05 seconds
        |_CBEventType.LLM -> 58.617289 seconds
**********
Cats are domesticated felines known for their independence, agility, and unique behaviors. They are members of the Felidae family and exhibit a remarkable array of traits, including a strong sense of territory, excellent hearing, and a distinctive purr. Cats are often portrayed as mysterious and independent creatures, though they are also beloved companions.
